In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [26]:
# Source Data from https://academictorrents.com/details/c398a571976c78d346c325bd75c47b82edf6124e
# Only the r/Progresspics subreddit

# Read the Data in
df = pd.read_json('progresspics_submissions', lines=True)

In [27]:
df.loc[df['title'].notna(), 'title'].count()

222645

In [28]:
# Extract A/S/L
df['created_date'] = pd.to_datetime(df['created_utc'],unit='s')
df[['gender', 'age', 'height']] = df['title'].str.extract(r'^([A-Z])/([0-9]+)/([^ ]*)')
df[['start_weight', 'end_weight', 'lost_weight']] = df['title'].str.extract(r'\[([0-9]{2,})\D*&gt;\D*([0-9]{2,})\D+([0-9]+)\D*\]')
df['age'] = df['age'].astype(float)

df['weight_unit'] = pd.NA

# Generate weight_unit
row_filter = df['title'].str.contains(r'\[.*lbs.*\]', regex=True) & (df['weight_unit'].isna())
df.loc[row_filter , 'weight_unit'] = 'lbs'
row_filter = df['title'].str.contains(r'\[.*kg.*\]', regex=True) & (df['weight_unit'].isna())
df.loc[row_filter , 'weight_unit'] = 'kg'
row_filter = df['title'].str.contains(r'\[.*kilos.*\]', regex=True) & (df['weight_unit'].isna())
df.loc[row_filter , 'weight_unit'] = 'kg'
row_filter = df['title'].str.contains(r'\[.*stone.*\]', regex=True) & (df['weight_unit'].isna())
df.loc[row_filter , 'weight_unit'] = 'stone'
row_filter = df['title'].str.contains(r'\[.*\bst\b.*\]', regex=True) & (df['weight_unit'].isna())
df.loc[row_filter , 'weight_unit'] = 'stone'

#if unspecified, make a guess based on ranges. 
row_filter = ((df['weight_unit'].isna()) & (df['start_weight'].astype(float) >= 100))
df.loc[row_filter , 'weight_unit'] = 'lbs'
row_filter = ((df['weight_unit'].isna()) & (df['start_weight'].astype(float).between(0, 20)))
df.loc[row_filter , 'weight_unit'] = 'stone'
row_filter = ((df['weight_unit'].isna()) & (df['start_weight'].astype(float).between(20, 100)))
df.loc[row_filter , 'weight_unit'] = 'kg'

In [30]:
# Convert Weights to lbs
new_columns = {'start_weight':'converted_start_weight', 'end_weight':'converted_end_weight', 'lost_weight':'converted_lost_weight'}

for column in new_columns:
    # convert if kgs
    row_filter = (df['weight_unit'] == 'kg') & (df[column].notnull()) 
    df.loc[row_filter, new_columns[column]] = df.loc[row_filter, column].astype(float) * 2.20462
    # convert if stone
    row_filter = (df['weight_unit'] == 'stone') & (df[column].notnull()) 
    df.loc[row_filter, new_columns[column]] = df.loc[row_filter, column].astype(float) * 14
    # convert if lbs
    row_filter = (df['weight_unit'] == 'lbs') & (df[column].notnull())
    df.loc[row_filter, new_columns[column]] = df.loc[row_filter, column].astype(float)

df['calculated_lost_weight'] = df['converted_start_weight'] - df['converted_end_weight']

In [31]:
# Extract Timeframes, clean data as needed
df['raw_timeframe'] = df['title'].str.extract(r'\[[0-9]{2,}\D*&gt;\D*[0-9]{2,}\D+[0-9]+\D*\][^\(]{0,5}\((.*?)[\)]')[0]  # Lazy quantifier courtesy of 46750
df['timeframe'] = df['raw_timeframe'].str.lower()

# Correct 1/2 to the best of my ability lol
df['timeframe'] = df['timeframe'].str.replace(' &amp; ', ' and ') # 46526
df['timeframe'] = df['timeframe'].str.replace(r'(\d+)( and|&amp; 1/2)', r'\1.5', regex=True).str.strip()    # 12249 or condition 13085
df['timeframe'] = df['timeframe'].str.replace(r'(\d+)(&amp;1/2)', r'\1.5', regex=True).str.strip()    #10882
df['timeframe'] = df['timeframe'].str.replace(r'(\d+)( 1/2)', r'\1.5', regex=True).str.strip() # 54949

row_filter = (df['raw_timeframe'].str.contains('\d', regex=True, na=False) & (df['timeframe'].str.len() < 20)) & (df['timeframe'].str.contains('year|month|week')) & df['timeframe'].str.contains('year|month|week', na=False)
df = df.loc[ row_filter]

In [32]:
df.loc[:, 'timeframe_years'] = df['timeframe'].str.lower().str.extract(r'\D*(\d+\.?\d*)\D?year|yr.*').rename(columns={0:'timeframe_years'}).astype(float) * 52
df.loc[:, 'timeframe_months'] = df['timeframe'].str.lower().str.extract(r'\D*(\d+\.?\d*)\D?month|mo.*').rename(columns={0:'timeframe_months'}).astype(float) * 4.3
df.loc[:, 'timeframe_weeks'] = df['timeframe'].str.lower().str.extract(r'\D*(\d+\.?\d*)\D?week.*').rename(columns={0:'timeframe_weeks'}).astype(float)
df.loc[:, 'timeframe_days'] = df['timeframe'].str.lower().str.extract(r'\D*(\d+\.?\d*)\D?day.*').rename(columns={0:'timeframe_days'}).astype(float) / 7

df['timeframe_in_weeks'] =  df[['timeframe_years','timeframe_months','timeframe_weeks', 'timeframe_days']].sum(axis=1)

df['loss_rate'] = (df['calculated_lost_weight'] / df['timeframe_in_weeks'])
df['loss_rate'] = df['loss_rate'].astype(float)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [33]:
#Reset the columns for debugging purposes
df['temp_height_inch'] = np.nan
df['temp_height_foot'] = np.nan
df['height_in_inches'] = np.nan

df['height'] = df['height'].str.lower().str.strip()

#Extract height in cm & convert
cm_filter = df['height'].str.contains('cm', na=False)
df.loc[cm_filter , 'height'] = df.loc[cm_filter , 'height'].str.replace('[.,]' , '', regex=True)
df.loc[cm_filter , 'height_in_inches'] = (df['height'].str.extract(r'(\d+)cm').astype(float) * 0.393701).rename(columns={0:'height_in_inches'})

#Extract height in ft / in
inch_filter = df['height'].str.contains('cm', na=True)
df[['temp_height_ft', 'temp_height_inch']] = df.loc[~inch_filter, 'height'].str.extract(r'\D*(\d+)\D*(\d+)\D*').astype(float)

#Clean up various iterations of just feet
df.loc[df['temp_height_ft'].isna(), 'temp_height_ft'] = df.loc[df['temp_height_ft'].isna(), 'height'].str.extract(r"\D*(\d+)\D*['`’]\D*").astype(float).rename(columns={0:'temp_height_ft'})
df.loc[df['temp_height_ft'].isna(), 'temp_height_ft'] = df.loc[df['temp_height_ft'].isna(), 'height'].str.extract(r"\D*(\d+)\D*ft\D*").astype(float).rename(columns={0:'temp_height_ft'})
df.loc[df['temp_height_ft'].isna(), 'temp_height_ft'] = df.loc[df['temp_height_ft'].isna(), 'height'].str.extract(r"\D*(\d+)\D*[\"\”]\D*").astype(float).rename(columns={0:'temp_height_ft'})
df.loc[df['temp_height_ft'].isna(), 'temp_height_ft'] = df.loc[df['temp_height_ft'].isna(), 'height'].str.extract(r"\D*([5-6]+)\D*").astype(float).rename(columns={0:'temp_height_ft'})

#If you listed ft but not inches, set inches to 0 for the summation
row_filter = (df['temp_height_ft'].notna()) & (df['temp_height_inch'].isna())
df.loc[row_filter, 'temp_height_inch'] = 0

#Calculate height_in_inches from ft + inches
df.loc[~inch_filter, 'height_in_inches'] = df.loc[~inch_filter , 'temp_height_ft'] * 12 + df.loc[~inch_filter ,'temp_height_inch']
filter = ((df['height_in_inches'].isna()) & (df['height'].notna()))

#df.loc[:, ['height', 'height_in_inches', 'temp_height_ft', 'temp_height_inch']]

In [34]:
# weight (lb) / [height (in)]^2 x 703
# From https://www.cdc.gov/nccdphp/dnpao/growthcharts/training/bmiage/page5_2.html
df['Starting_BMI'] = df['converted_start_weight'] / df['height_in_inches']**2 * 703
df['Ending_BMI'] = df['converted_end_weight'] / df['height_in_inches']**2 * 703

In [35]:
row_filter = (df['Starting_BMI'].notna()) & (df['loss_rate'].notna()) & (df['height_in_inches'].between(36,100)) & (df['calculated_lost_weight'].between(0,500))
#df.loc[row_filter,['Starting_BMI','height', 'height_in_inches', 'converted_start_weight', 'loss_rate']]
pd.options.display.max_colwidth = 390
df.loc[row_filter,['Starting_BMI','height', 'height_in_inches','start_weight', 'converted_start_weight', 'loss_rate', 'calculated_lost_weight', 'weight_unit', 'title']].sort_values(by='Starting_BMI', ascending=False)

,Starting_BMI,height,height_in_inches,start_weight,converted_start_weight,loss_rate,calculated_lost_weight,weight_unit,title
70217,119.422216,5'2'',62.0,653,653.00000,9.018088,349.00000,lbs,M/28/5'2'' [653lbs&gt;304lbs=349lbs] (9 months) - Loving the new me.
190932,117.166667,5’0”,60.0,600,600.00000,9.302326,200.00000,lbs,F/18/5’0” [600 &gt; 400 = 0] (5 months) Those numbers are a complete lie giving my real weight makes me gag but anyway. Progress !!!
170742,103.328521,"5'5""",65.0,621,621.00000,6.511628,70.00000,lbs,"F/32/5'5"" [621lbs &gt; 551lbs = 70lbs] (2.5 months) I have a lot of health problems and have recently been considered terminal. I changed my diet and started an increased dose of diuretics and have lost 70 lbs since Feb! Trying to prolong my life!"
54949,103.078287,"5'8""",68.0,678,678.00000,3.852080,500.00000,lbs,"M/42/5'8"" [678lbs&gt;178lbs = 500lbs] (2 years 6 months) Weight loss progress"
129081,98.170414,"5'5""",65.0,590,590.00000,2.211538,115.00000,lbs,"F/31/5'5"" [590LBS&gt; 475LBS = 115 lbs] (1 YEAR) Dropped my abusive ex and then healthily dropped some serious poundage. Still working on it! Found a picture that let me do a 1 year before/after"
...,...,...,...,...,...,...,...,...,...
29825,8.070214,"5'5""",65.0,22,48.50164,0.296776,15.43234,kg,"F/22/5'5"" [22% BF &gt; 15% BF = 7% BF](1 YEAR) Lost the extra fat, and toned up! :)"
169707,6.583767,"5'2""",62.0,36,36.00000,0.007692,2.00000,lbs,"F/24/5'2"" [36M &gt; 34I = -7lbs] (5 years on) Breast Reduction *warning includes nipples*"
70084,0.000000,"5'5""",65.0,00,0.00000,0.000000,0.00000,lbs,"F/26/5'5"" [00lbs &gt; 00lbs = 00lbs] (4 years) Face progress after recovering from bulimia. Omitted weight for obvious reasons, I don't know it!"
140182,0.000000,5’2”,62.0,000,0.00000,0.000000,0.00000,stone,"F/30/5’2” [000&gt;000=000] (3 years) I believe this is considered “human progress” per the rules! Lifelong nail &amp; cuticle picker, proud of my presentable hands now 💅🏻"


In [36]:
# Drop Temporary Columns
df = df.drop(columns=['timeframe_years','timeframe_months','timeframe_weeks', 'timeframe_days', 'converted_lost_weight', 'raw_timeframe', 'temp_height_inch', 'temp_height_foot', 'temp_height_ft'])

In [38]:
df.to_pickle('df.pkl')
